In [1]:
import chess.pgn
import chess.engine
import chess.svg

import logging
from pprint import pprint
import matplotlib.pyplot as plt
import pickle as pkl

from utils import who, reduced_fen

## PARAMS
display_board = False
limit = chess.engine.Limit(time = 5) #time = 5
user = 'datashrimp'
move_limit = 5
logging_level = logging.WARNING # logging.INFO # logging.WARNING
pgn_file = "./data/{}.pgn".format(user)
max_games = 100000
load = False
save = True

## RESET PGN READER

if load:
    counter = pkl.load(open('./store/{}/counter_{}.pkl'.format(user, move_limit), 'rb'))
    analysis = pkl.load(open('./store/{}/analysis_{}.pkl'.format(user, move_limit), 'rb'))
    fen_info = pkl.load(open('./store/fen_info_{}.pkl'.format(move_limit), 'rb'))
else:
    counter = 0
    analysis = {}
    fen_info = {}
    
pgn = open(pgn_file)
for i in range(counter):
    chess.pgn.skip_game(pgn)

logging.getLogger().setLevel(logging_level)



In [ ]:

while counter < max_games:
    
    engine = chess.engine.SimpleEngine.popen_uci("./stockfish/Mac/stockfish-10-bmi2")

    game = chess.pgn.read_game(pgn)
    board = game.board()
    
    result = game.headers['Result']
    new_pos_counter = 0
    
    if game.headers['White'] == user:
        user_colour = True
        if result == '1-0':
            user_result = 1
        if result == '0-1':
            user_result = 0
        logging.info("User is White")
    else:
        user_colour = False
        if result == '0-1':
            user_result = 1
        if result == '1-0':
            user_result = 0
        logging.info("User is Black")
        
    if result == '1/2-1/2':
        user_result = 0.5

    if result != '*':

        logging.info("Initial board")

        for move in game.mainline_moves():
            
            
            
            if board.fullmove_number <= move_limit:

                fen = reduced_fen(board.fen())
                san = board.san(move)

                logging.info("{}:{}:{}".format(board.fullmove_number, who(board.turn),  san))

                user_move = board.turn == user_colour

                if user_move:

                    if fen not in fen_info:
                        info = engine.analyse(board, limit) #, multipv = 4
                        fen_info[fen] = info
                    else:
                        info = fen_info[fen]

                    before_score = info["score"].pov(user_colour).score(mate_score=1000)
                    best_move = board.san(info['pv'][0])

                    logging.info("Best move:{}".format(best_move))
                    logging.info("Score before move:{}".format(before_score))

                if display_board:
                    display(chess.svg.board(board, flipped = not user_colour, lastmove = move, size = 400))

                board.push(move)

                if user_move:
                    new_fen = reduced_fen(board.fen())
                    if new_fen not in fen_info:
                        new_pos_counter += 1
                        
                        info = engine.analyse(board, limit) #, multipv = 4
                        fen_info[new_fen] = info
                    else:
                        info = fen_info[new_fen]

                    after_score = info["score"].pov(user_colour).score(mate_score=1000)
                    logging.info("Score after move:{}".format(after_score))
                    diff = after_score - before_score
                    logging.info("Score diff:{}".format(diff))

                    if (fen, san) not in analysis:
                        analysis[(fen, san)] = {'count': 0, 'colour': user_colour
                                                , 'move': san, 'fen': fen, 'diff': diff
                                                , 'best_move': best_move, 'before_score': before_score
                                               , 'after_score': after_score
                                               , 'result': 0
                                               , 'win_perc': 0}

                    analysis[(fen, san)]['count'] += 1
                    analysis[(fen, san)]['result'] += user_result
                    analysis[(fen, san)]['win_perc'] = (analysis[(fen, san)]['result']+0.0) / analysis[(fen, san)]['count']

                logging.info("")

            else:
    #             pprint(analysis)
                break
            
    counter += 1
    if save:
        pkl.dump(analysis, open('./store/{}/analysis_{}.pkl'.format(user, move_limit), 'wb'))
        pkl.dump(fen_info, open('./store/fen_info_{}.pkl'.format(move_limit), 'wb'))
        pkl.dump(counter, open('./store/{}/counter_{}.pkl'.format(user, move_limit), 'wb'))
    
    print('{} new positions analysed'.format(new_pos_counter))
    print("{} games logged".format(counter))

5 new positions analysed
1 games logged
4 new positions analysed
2 games logged
3 new positions analysed
3 games logged
4 new positions analysed
4 games logged
3 new positions analysed
5 games logged
5 new positions analysed
6 games logged
3 new positions analysed
7 games logged
4 new positions analysed
8 games logged
2 new positions analysed
9 games logged
3 new positions analysed
10 games logged
3 new positions analysed
11 games logged
2 new positions analysed
12 games logged
2 new positions analysed
13 games logged
3 new positions analysed
14 games logged
2 new positions analysed
15 games logged
1 new positions analysed
16 games logged
3 new positions analysed
17 games logged
4 new positions analysed
18 games logged
3 new positions analysed
19 games logged
4 new positions analysed
20 games logged
2 new positions analysed
21 games logged
3 new positions analysed
22 games logged
5 new positions analysed
23 games logged
2 new positions analysed
24 games logged
1 new positions analysed


4 new positions analysed
199 games logged
2 new positions analysed
200 games logged
3 new positions analysed
201 games logged
3 new positions analysed
202 games logged
2 new positions analysed
203 games logged
1 new positions analysed
204 games logged
4 new positions analysed
205 games logged
5 new positions analysed
206 games logged
1 new positions analysed
207 games logged
3 new positions analysed
208 games logged
2 new positions analysed
209 games logged
4 new positions analysed
210 games logged
0 new positions analysed
211 games logged
1 new positions analysed
212 games logged
1 new positions analysed
213 games logged
4 new positions analysed
214 games logged
5 new positions analysed
215 games logged
4 new positions analysed
216 games logged
3 new positions analysed
217 games logged
1 new positions analysed
218 games logged
0 new positions analysed
219 games logged
0 new positions analysed
220 games logged
2 new positions analysed
221 games logged
4 new positions analysed
222 games

0 new positions analysed
395 games logged
4 new positions analysed
396 games logged
2 new positions analysed
397 games logged
2 new positions analysed
398 games logged
1 new positions analysed
399 games logged
0 new positions analysed
400 games logged
4 new positions analysed
401 games logged
1 new positions analysed
402 games logged
3 new positions analysed
403 games logged
2 new positions analysed
404 games logged
1 new positions analysed
405 games logged
3 new positions analysed
406 games logged
4 new positions analysed
407 games logged
3 new positions analysed
408 games logged
2 new positions analysed
409 games logged
2 new positions analysed
410 games logged
2 new positions analysed
411 games logged
3 new positions analysed
412 games logged
3 new positions analysed
413 games logged
1 new positions analysed
414 games logged
4 new positions analysed
415 games logged
2 new positions analysed
416 games logged
0 new positions analysed
417 games logged
2 new positions analysed
418 games

1 new positions analysed
591 games logged
3 new positions analysed
592 games logged
1 new positions analysed
593 games logged
3 new positions analysed
594 games logged
0 new positions analysed
595 games logged
2 new positions analysed
596 games logged
1 new positions analysed
597 games logged
2 new positions analysed
598 games logged
1 new positions analysed
599 games logged
0 new positions analysed
600 games logged
2 new positions analysed
601 games logged
0 new positions analysed
602 games logged
0 new positions analysed
603 games logged
2 new positions analysed
604 games logged
0 new positions analysed
605 games logged
1 new positions analysed
606 games logged
0 new positions analysed
607 games logged
2 new positions analysed
608 games logged
0 new positions analysed
609 games logged
2 new positions analysed
610 games logged
1 new positions analysed
611 games logged
3 new positions analysed
612 games logged
1 new positions analysed
613 games logged
2 new positions analysed
614 games

2 new positions analysed
787 games logged
1 new positions analysed
788 games logged
2 new positions analysed
789 games logged
2 new positions analysed
790 games logged
2 new positions analysed
791 games logged
5 new positions analysed
792 games logged
2 new positions analysed
793 games logged
1 new positions analysed
794 games logged
0 new positions analysed
795 games logged
5 new positions analysed
796 games logged
0 new positions analysed
797 games logged
0 new positions analysed
798 games logged
1 new positions analysed
799 games logged
3 new positions analysed
800 games logged
0 new positions analysed
801 games logged
1 new positions analysed
802 games logged
3 new positions analysed
803 games logged
1 new positions analysed
804 games logged
3 new positions analysed
805 games logged
0 new positions analysed
806 games logged
3 new positions analysed
807 games logged
4 new positions analysed
808 games logged
4 new positions analysed
809 games logged
0 new positions analysed
810 games

In [ ]:
analysis